In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
# %matplotlib inline
%matplotlib qt5
import mne
mne.viz.set_browser_backend("qt")  # or "matplotlib"
mne.set_config("MNE_BROWSER_BACKEND", "qt")  # or "matplotlib"
%gui qt

import IPython

# Jupyter-lab enable printing for any line on its own (instead of just the last one in the cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


Automatic pdb calling has been turned OFF
Using qt as 2D backend.



# Use MNE to load and analyze saved EEG and Motion recordings


In [8]:
import time
import re
from datetime import datetime, timezone

import uuid
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from nptyping import NDArray
from matplotlib import pyplot as plt

from pathlib import Path
import numpy as np
import pandas as pd
from numpy.typing import NDArray

import mne
from mne import set_log_level
from copy import deepcopy
import mne

from mne.io import read_raw

datasets = []
# mne.viz.set_browser_backend("Matplotlib")
mne.viz.set_browser_backend("qt")

from mne_lsl.player import PlayerLSL as Player
from mne_lsl.stream import StreamLSL as Stream

from phoofflineeeganalysis.analysis.MNE_helpers import MNEHelpers
from phoofflineeeganalysis.analysis.historical_data import HistoricalData
from phoofflineeeganalysis.analysis.motion_data import MotionData
from phoofflineeeganalysis.analysis.EEG_data import EEGComputations, EEGData
from phoofflineeeganalysis.analysis.anatomy_and_electrodes import ElectrodeHelper
# from ..EegProcessing import bandpower
# from phoofflineeeganalysis.EegProcessing import analyze_eeg_trends
from phoofflineeeganalysis.EegVisualization import VisHelpers
from phoofflineeeganalysis.analysis.SavedSessionsProcessor import SavedSessionsProcessor, SessionModality, DataModalityType

set_log_level("WARNING")


# db_root_path = Path('/content/drive/MyDrive/Databases').resolve()
db_root_path = Path(r'E:/Dropbox (Personal)/Databases').resolve()
assert db_root_path.exists(), f"'{db_root_path.as_posix()}' does not exist!"

# eeg_recordings_file_path: Path = Path(r'E:/Dropbox (Personal)/Databases/UnparsedData/EmotivEpocX_EEGRecordings/fif').resolve()
# headset_motion_recordings_file_path: Path = Path(r'E:/Dropbox (Personal)/Databases/UnparsedData/EmotivEpocX_EEGRecordings/MOTION_RECORDINGS/fif').resolve()

# assert eeg_recordings_file_path.exists()
# assert headset_motion_recordings_file_path.exists()

eeg_recordings_file_path: Path = db_root_path.joinpath('UnparsedData/EmotivEpocX_EEGRecordings/fif').resolve()
flutter_eeg_recordings_file_path: Path = db_root_path.joinpath('UnparsedData/EmotivEEG_FlutterRecordings').resolve()
flutter_motion_recordings_file_path: Path = db_root_path.joinpath('UnparsedData/EmotivEEG_FlutterRecordings/MOTION_RECORDINGS').resolve()
flutter_GENERIC_recordings_file_path: Path = db_root_path.joinpath('UnparsedData/EmotivEEG_FlutterRecordings/GENERIC_RECORDINGS').resolve()

headset_motion_recordings_file_path: Path = db_root_path.joinpath('UnparsedData/EmotivEpocX_EEGRecordings/MOTION_RECORDINGS/fif').resolve()
WhisperVideoTranscripts_LSL_Converted = db_root_path.joinpath('UnparsedData/WhisperVideoTranscripts_LSL_Converted').resolve()
pho_log_to_LSL_recordings_path: Path = db_root_path.joinpath('UnparsedData/PhoLogToLabStreamingLayer_logs').resolve()
## These contain little LSL .fif files with names like: '20250808_062814_log.fif',

eeg_analyzed_parent_export_path = db_root_path.joinpath('AnalysisData/MNE_preprocessed').resolve()
pickled_data_path = db_root_path.joinpath('AnalysisData/MNE_preprocessed/PICKLED_COLLECTION').resolve()
assert pickled_data_path.exists()


# n_most_recent_sessions_to_preprocess: int = None # None means all sessions
# n_most_recent_sessions_to_preprocess: int = 35
# n_most_recent_sessions_to_preprocess: int = 5
n_most_recent_sessions_to_preprocess = None


'qt'

# SavedSessionProcessor

In [3]:

sso: SavedSessionsProcessor = SavedSessionsProcessor(eeg_recordings_file_path=eeg_recordings_file_path,
                                                     headset_motion_recordings_file_path=headset_motion_recordings_file_path, WhisperVideoTranscripts_LSL_Converted_file_path=WhisperVideoTranscripts_LSL_Converted, pho_log_to_LSL_recordings_path=pho_log_to_LSL_recordings_path,
                                                    eeg_analyzed_parent_export_path=eeg_analyzed_parent_export_path, 
                                                     n_most_recent_sessions_to_preprocess=n_most_recent_sessions_to_preprocess, 
                                                    should_load_data=True, should_load_preprocessed=False,
                                                    #  should_load_data=True, should_load_preprocessed=True,
													)

# 2025-09-18 - LabRecorder XDF Imports

In [4]:
from phoofflineeeganalysis.analysis.EEG_data import EEGData
from phoofflineeeganalysis.analysis.MNE_helpers import DatasetDatetimeBoundsRenderingMixin, RawArrayExtended, RawExtended, up_convert_raw_objects, up_convert_raw_obj
from phoofflineeeganalysis.analysis.SavedSessionsProcessor import LabRecorderXDF, unwrap_single_element_listlike_if_needed


lab_recorder_output_path = Path(r"E:\Dropbox (Personal)\Databases\UnparsedData\LabRecorderStudies\sub-P001").resolve()
assert lab_recorder_output_path.exists()

labRecorder_PostProcessed_path: Path = sso.eeg_analyzed_parent_export_path.joinpath(f'LabRecorder_PostProcessed')
labRecorder_PostProcessed_path.mkdir(exist_ok=True)

should_write_final_merged_eeg_fif: bool = True
# should_write_final_merged_eeg_fif: bool = False
_out_eeg_raw, _out_xdf_stream_infos_df, lab_recorder_xdf_files = LabRecorderXDF.load_and_process_all(lab_recorder_output_path=lab_recorder_output_path, 
                                                                                                     labRecorder_PostProcessed_path=labRecorder_PostProcessed_path, should_write_final_merged_eeg_fif=should_write_final_merged_eeg_fif)
xdf_dataset_indicies = np.unique(deepcopy(_out_xdf_stream_infos_df).reset_index(drop=False, inplace=False)['xdf_dataset_idx'].to_numpy())
n_unique_xdf_datasets: int = len(xdf_dataset_indicies)
print(f'n_unique_xdf_datasets: {n_unique_xdf_datasets}')


trying to process XDF file 0/39: "E:/Dropbox (Personal)/Databases/UnparsedData/LabRecorderStudies/sub-P001/LabRecorder_2025-09-10T140840.440Z_eeg.xdf"...
	 failed with error: 1
	skipping file.
trying to process XDF file 1/39: "E:/Dropbox (Personal)/Databases/UnparsedData/LabRecorderStudies/sub-P001/LabRecorder_2025-09-10T153731.079Z_eeg.xdf"...


Stream 3: Calculated effective sampling rate 32.0162 Hz is different from specified rate 16.0000 Hz.


saving finalized EEG data out to "E:/Dropbox (Personal)/Databases/AnalysisData/MNE_preprocessed/LabRecorder_PostProcessed/2025-09-10T15-37-31.fif"


C:\Users\pho\repos\EmotivEpoc\PhoOfflineEEGAnalysis\src\phoofflineeeganalysis\analysis\SavedSessionsProcessor.py:981: RuntimeWarning: This filename (E:\Dropbox (Personal)\Databases\AnalysisData\MNE_preprocessed\LabRecorder_PostProcessed\2025-09-10T15-37-31.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_raw.save(a_lab_recorder_filepath, overwrite=True)


trying to process XDF file 2/39: "E:/Dropbox (Personal)/Databases/UnparsedData/LabRecorderStudies/sub-P001/LabRecorder_2025-09-10T165236.402Z_eeg.xdf"...


...

trying to process XDF file 37/39: "E:/Dropbox (Personal)/Databases/UnparsedData/LabRecorderStudies/sub-P001/LabRecorder_rMBP_2025-09-11T203807.731Z.xdf"...


Stream 3: Calculated effective sampling rate 32.0160 Hz is different from specified rate 16.0000 Hz.


	 failed with error: not enough values to unpack (expected 2, got 1)
	skipping file.
trying to process XDF file 38/39: "E:/Dropbox (Personal)/Databases/UnparsedData/LabRecorderStudies/sub-P001/LabRecorder_rMBP_2025-09-11T210420.828Z.xdf"...


Stream 4: Calculated effective sampling rate 32.0167 Hz is different from specified rate 16.0000 Hz.
Stream 5: Calculated effective sampling rate 32.0166 Hz is different from specified rate 16.0000 Hz.


	 failed with error: orig_time should be the same to add/concatenate 2 annotations (got 2025-09-11 21:04:20+00:00 != None)
	skipping file.
n_unique_xdf_datasets: 25


In [ ]:
from phoofflineeeganalysis.analysis.SavedSessionsProcessor import XDFDataStreamAccessor


_out_xdf_stream_infos_df: pd.DataFrame = XDFDataStreamAccessor.init_from_results(_out_xdf_stream_infos_df=_out_xdf_stream_infos_df, active_only_out_eeg_raws=_out_eeg_raw) # [_out_xdf_stream_infos_df['name'] == 'Epoc X']
_out_xdf_stream_infos_df

In [ ]:
# from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import Spike2DRaster, SynchronizedPlotMode

In [ ]:
_out_xdf_stream_infos_df

In [ ]:
len(_out_xdf_stream_infos_df)
_out_xdf_stream_infos_df.loc[0]

In [ ]:
_out_eeg_raw

# Run Batch Computations on `_out_eeg_raw`

In [ ]:
num_sessions: int = len(_out_eeg_raw)
num_sessions


results_dict = {}

for an_xdf_dataset_idx in np.arange(num_sessions):
    a_raw = deepcopy(_out_eeg_raw[an_xdf_dataset_idx])
    a_raw.down_convert_to_base_type()
    results_dict[an_xdf_dataset_idx] 

## Common Setup

In [ ]:
a_raw = deepcopy(_out_eeg_raw[-3])
a_raw.down_convert_to_base_type()

In [ ]:
## INPUT: fixed_len_epochs
freqs = np.arange(5., 40., 1.0)
# Define frequencies and number of cycles
# freqs = np.logspace(*np.log10([2, 40]), num=20)
n_cycles = freqs / 2.0 # A common approach is to use a fixed number of cycles or a value that increases with frequency.

freqs
n_cycles

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.BinnedImageRenderingWindow import BasicBinnedImageRenderingWindow, LayoutScrollability


### Fixed Length Epoch-based:

In [ ]:
from phoofflineeeganalysis.analysis.MNE_helpers import RawExtended, RawArrayExtended

## INPUT: a_raw
fixed_len_epochs = mne.make_fixed_length_epochs(a_raw, duration=8, preload=True, reject_by_annotation=False)
fixed_len_epochs
	

In [ ]:
## INPUT: fixed_len_epochs
freqs = np.arange(5., 40., 1.0)

# Define frequencies and number of cycles
# freqs = np.logspace(*np.log10([2, 40]), num=20)
n_cycles = freqs / 2.0 # A common approach is to use a fixed number of cycles or a value that increases with frequency.

# n_cycles = np.linspace(3,10,len(freqs))


In [ ]:


# Compute time-frequency power using Morlet wavelets -- NO EPOCHS, RAW DATA
# power = a_raw.compute_tfr(method="morlet",
power = mne.time_frequency.tfr_morlet(
    fixed_len_epochs,
    freqs=freqs,
    n_cycles=n_cycles,
    return_itc=False,
    average=False, # Compute TFR on continuous data without averaging
    # average=True, # Compute TFR on continuous data without averaging
    decim=3, # Decimate the output for faster computation and smaller file size
    n_jobs=-1 # Use 1 core for this example
)
power

In [ ]:
## Array version:
from mne.time_frequency import tfr_array_morlet

# get raw data (channels × time)
data = fixed_len_epochs.get_data() # shape: (n_epochs, n_channels, n_times)
print(f'np.shape(data): {np.shape(data)}')
# run morlet on continuous data
power: NDArray = tfr_array_morlet(
    data=data,
    sfreq=fixed_len_epochs.info['sfreq'],
    freqs=freqs,
    n_cycles=n_cycles,
    output='power',
    decim=3,
    n_jobs=-1
)

# power shape: (n_epochs=1, n_channels, n_freqs, n_times)
power = np.squeeze(power) ## remove epoch
print(f'np.shape(power): {np.shape(power)}')

# np.shape(data): (593, 14, 1024)
# np.shape(power): (593, 14, 20, 342)

In [ ]:
import h5py

repo_data_folder_path = Path(r"C:\Users\pho\repos\EmotivEpoc\PhoLabStreamingReceiver\data").resolve()

h5_file_path = repo_data_folder_path.joinpath(f'2025-09-21_exported_sess.h5').resolve()

print(f'writing to "{h5_file_path.as_posix()}"')

# raw_data = a_raw.get_data() # (14, 607857)
# np.shape(raw_data)


In [ ]:
with h5py.File(h5_file_path, "w") as f:
    
    d0 = f.create_dataset("raw", data=raw_data)
    d0.attrs["dim_labels"] = ["channels", "time"]

    d1 = f.create_dataset("epoched", data=data)
    d1.attrs["dim_labels"] = ["epochs", "channels", "time"]

    d2 = f.create_dataset("power", data=power)
    d2.attrs["dim_labels"] = ["epochs", "channels", "time", "frequency"]


print(f'done! wrote to "{h5_file_path.as_uri()}"')

# ACTIVE: refine the active raws and compute them

In [5]:
from phoofflineeeganalysis.analysis.EEG_data import EEGComputations, EEGData
from phoofflineeeganalysis.PendingNotebookCode import batch_compute_all_eeg_datasets, render_all_spectograms_to_high_quality_pdfs, plot_all_spectograms
from phoofflineeeganalysis.PendingNotebookCode import plot_session_spectogram

In [6]:
## INPUTS: _out_eeg_raw
# Process only the last 5 datasets using 4 workers:
active_only_out_eeg_raws, results = batch_compute_all_eeg_datasets(eeg_raws=_out_eeg_raw, limit_num_items=150, max_workers = 4)

## OUTPUTS: active_only_out_eeg_raws, results
# 1m 19.8s for 25 sessions

Processing 25 EEG datasets using 4 parallel workers...
  Processing dataset 1/25 (meas_date: 2025-09-10 15:37:31+00:00)
running raw_data_topo...
  Processing dataset 2/25 (meas_date: 2025-09-11 01:41:54+00:00)
running raw_data_topo...
  Processing dataset 3/25 (meas_date: 2025-09-11 10:13:28+00:00)
running raw_data_topo...
  Processing dataset 4/25 (meas_date: 2025-09-11 15:45:49+00:00)
running raw_data_topo...
for INPUT PARAMS: epoch_dur: 4, epoch_step: 0.25, moving_avg_epochs: 32
	step_samples: 32, window_samples: 512
	n_ch: 14, n_times: 21801
	done.
running cwt...
	done.
running spectogram...
	done.
  Completed dataset 2/25 (meas_date: 2025-09-11 01:41:54+00:00)
  Processing dataset 5/25 (meas_date: 2025-09-11 21:36:35+00:00)
running raw_data_topo...
for INPUT PARAMS: epoch_dur: 4, epoch_step: 0.25, moving_avg_epochs: 32
	step_samples: 32, window_samples: 512
	n_ch: 14, n_times: 5590
	done.
running cwt...
	done.
running spectogram...
	done.
  Completed dataset 5/25 (meas_date: 2025-

In [ ]:
from phoofflineeeganalysis.analysis.SavedSessionsProcessor import XDFDataStreamAccessor

num_sessions: int = len(results)
num_sessions

# xdf_stream_infos_df: pd.DataFrame = XDFDataStreamAccessor.init_from_results(_out_xdf_stream_infos_df=_out_xdf_stream_infos_df, active_only_out_eeg_raws=active_only_out_eeg_raws)
xdf_stream_infos_df: pd.DataFrame = XDFDataStreamAccessor.init_from_results(_out_xdf_stream_infos_df=_out_xdf_stream_infos_df, active_only_out_eeg_raws=active_only_out_eeg_raws)
# xdf_stream_infos_df: pd.DataFrame = XDFDataStreamAccessor.init_from_results(_out_xdf_stream_infos_df=_out_xdf_stream_infos_df[_out_xdf_stream_infos_df['name'] == 'Epoc X'], active_only_out_eeg_raws=active_only_out_eeg_raws)
xdf_stream_infos_df

In [ ]:
from phoofflineeeganalysis.analysis.EEG_data import EEGComputations
from phoofflineeeganalysis.analysis.SavedSessionsProcessor import LabRecorderXDF

hdf5_out_path: Path = Path('E:/Dropbox (Personal)/Databases/AnalysisData/MNE_preprocessed').joinpath('2025-09-29_all_HDF.h5').resolve()

hdf5_out_path.unlink(missing_ok=True)

LabRecorderXDF.to_hdf(active_only_out_eeg_raws=active_only_out_eeg_raws, results=results, xdf_stream_infos_df=xdf_stream_infos_df, file_path=hdf5_out_path, root_key='/')


In [ ]:
from phoofflineeeganalysis.analysis.EEG_data import EEGComputations, EEGData

write_mode = 'r+'
if (not hdf5_out_path.exists()):
    write_mode = 'w'
    
write_mode = 'w'

with h5py.File(hdf5_out_path, write_mode) as f:
    for an_xdf_dataset_idx in np.arange(num_sessions):
        if (an_xdf_dataset_idx < 5):
            a_raw = active_only_out_eeg_raws[an_xdf_dataset_idx]
            a_meas_date = a_raw.info.get('meas_date')
            a_raw_key: str = a_meas_date.strftime("%Y-%m-%d/%H-%M-%S") # '2025-09-22/21-35-47'
            a_result = results[an_xdf_dataset_idx]
            # EEGComputations.to_hdf(a_result=a_result, file_path=hdf5_out_path, root_key=f'/result/{a_raw_key}')
            # EEGComputations.to_hdf(a_result=a_result, file_path=f, root_key=f'/result/{a_raw_key}')
            EEGComputations.perform_write_to_hdf(a_result=a_result, f=f, root_key=f'/result/{a_raw_key}')
            # a_stream_info = deepcopy(_out_xdf_stream_infos_df).loc[an_xdf_dataset_idx]    
            # print(f'i: {i}, a_meas_date: {a_meas_date}, a_stream_info: {a_stream_info}\n\n')
            # print(f'i: {an_xdf_dataset_idx}, a_meas_date: {a_meas_date}')
            # a_df = a_raw.annotations.to_data_frame(time_format='datetime')
            # a_df = a_df[a_df['description'] != 'BAD_motion']
            # a_df['xdf_dataset_idx'] = an_xdf_dataset_idx
            # flat_annotations.append(a_df)

# flat_annotations = pd.concat(flat_annotations, ignore_index=True)
# flat_annotations

In [ ]:
flat_annotations['onset_str'] = flat_annotations['onset'].dt.strftime("%Y-%m-%d_%I:%M:%S.%f %p")
flat_annotations

# Out to HDF5

In [ ]:
import h5py
from phoofflineeeganalysis.analysis.EEG_data import EEGComputations

hdf5_out_path: Path = Path('E:/Dropbox (Personal)/Databases/AnalysisData/MNE_preprocessed').joinpath('2025-09-29_all_HDF.h5').resolve()
hdf5_out_path


In [ ]:
import h5py

_loaded_dict = {'xdf_stream_infos_df': None, 'flat_annotations': None, }

_loaded_dict['xdf_stream_infos_df'] = pd.read_hdf(hdf5_out_path, mode='r', key='/xdf_stream_infos_df')
_loaded_dict['flat_annotations'] = pd.read_hdf(hdf5_out_path, mode='r', key='/flat_annotations')
_loaded_dict


In [ ]:
with h5py.File(hdf5_out_path, 'r') as f:
    # f.attrs
    # f.name
    # f.visititems()
    _loaded_dict['xdf_stream_infos_df'] = pd.read_hdf(f, key='/xdf_stream_infos_df')
    

In [ ]:
xdf_stream_infos_df.to_hdf(hdf5_out_path, key='/xdf_stream_infos_df', append=False)
flat_annotations.to_hdf(hdf5_out_path, key='/flat_annotations_df', append=True)


In [ ]:
for an_xdf_dataset_idx in np.arange(num_sessions):
    a_raw = active_only_out_eeg_raws[an_xdf_dataset_idx]
    a_meas_date: datetime = a_raw.info.get('meas_date')
    a_result = results[an_xdf_dataset_idx]
    # a_stream_info = deepcopy(_out_xdf_stream_infos_df).loc[an_xdf_dataset_idx]    
    # print(f'i: {i}, a_meas_date: {a_meas_date}, a_stream_info: {a_stream_info}\n\n')
    # print(f'i: {an_xdf_dataset_idx}, a_meas_date: {a_meas_date}')
    # a_df = a_raw.to_data_frame(time_format='datetime')

    a_raw_key: str = a_meas_date.strftime("%Y-%m-%d/%H-%M-%S") # '2025-09-22/21-35-47'
    a_raw.to_data_frame(time_format='datetime').to_hdf(hdf5_out_path, key=f'/raw/{a_raw_key}/df', append=True)
    EEGComputations.to_hdf(a_result=a_result, file_path=hdf5_out_path, root_key=f'/result/{a_raw_key}')
    


In [ ]:
# a_raw_key: str = a_meas_date.strftime("%Y-%m-%dT%H-%M-%S")
a_raw_key: str = a_meas_date.strftime("%Y-%m-%d/%H-%M-%S") # '2025-09-22/21-35-47'
a_raw.to_data_frame(time_format='datetime').to_hdf(hdf5_out_path, key=f'/raw/{a_raw_key}/df', append=True)

EEGComputations.to_hdf(a_result=a_result, file_path=hdf5_out_path, root_key=f'/result/{a_raw_key}')



In [ ]:
a_result

In [ ]:



all_WHISPER_df.drop(columns=['filepath']).to_hdf(hdf5_out_path, key='modalities/WHISPER/df', append=True)
all_pho_log_to_lsl_df.drop(columns=['filepath']).to_hdf(hdf5_out_path, key='modalities/PHO_LOG_TO_LSL/df', append=True)


# Spike3DRasterWindowWidget

In [ ]:
# from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import Spike2DRaster
# from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _setup_spike_raster_window_for_debugging
# from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import ScatterItemData
from pyphoplacecellanalysis.GUI.Qt.SpikeRasterWindows.Spike3DRasterWindowWidget import Spike3DRasterWindowWidget


In [ ]:
## It's passed a specific computation_result which has a .sess attribute that's used to determine which spikes are displayed or not.
spike_raster_window: Spike3DRasterWindowWidget = Spike3DRasterWindowWidget(spikes_df, type_of_3d_plotter=type_of_3d_plotter, application_name=f'Spike Raster Window - {active_display_fn_identifying_ctx_string}', neuron_colors=neuron_colors, neuron_sort_order=neuron_sort_order,
                                                                            params_kwargs=dict(use_docked_pyqtgraph_plots=use_docked_pyqtgraph_plots),
                                                                            ) ## surprisingly only needs spikes_df !!?!
# Set Window Title Options:
a_file_prefix = str(computation_result.sess.filePrefix.resolve())
spike_raster_window.setWindowFilePath(a_file_prefix)
spike_raster_window.setWindowTitle(f'Spike Raster Window - {active_config_name} - {a_file_prefix}')


In [ ]:

# Gets the existing SpikeRasterWindow or creates a new one if one doesn't already exist:
# spike_raster_window, (active_2d_plot, active_3d_plot, main_graphics_layout_widget, main_plot_widget, background_static_scroll_plot_widget) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=True, allow_replace_hardcoded_main_plots_with_tracks=True)
# spike_raster_window, (active_2d_plot, active_3d_plot, main_graphics_layout_widget, main_plot_widget, background_static_scroll_plot_widget) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=False, allow_replace_hardcoded_main_plots_with_tracks=True)
# spike_raster_window, (active_2d_plot, active_3d_plot, *_all_outputs_dict) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=False, allow_replace_hardcoded_main_plots_with_tracks=True)
spike_raster_window, (active_2d_plot, active_3d_plot, *_all_outputs_dict) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=True, allow_replace_hardcoded_main_plots_with_tracks=True)

# CustomCalendarWidget

In [ ]:
from datetime import datetime, timezone
from phoofflineeeganalysis.UI.CustomCalendarWidget import CalendarDatasource
from phoofflineeeganalysis.UI.CustomCalendarWidget import CustomDataDisplayingCalendar

a_ds: CalendarDatasource = CalendarDatasource(xdf_stream_infos_df=xdf_stream_infos_df)
ex = CustomDataDisplayingCalendar()
ex.show()
ex.set_datasource(data_source=a_ds)

In [ ]:
found_df = a_ds.get_records_for_day(day_date=datetime(2025, 9, 10, tzinfo=timezone.utc))
found_df

In [ ]:
day_date = datetime(2025, 9, 11, tzinfo=timezone.utc)
day_date = day_date.replace(hour=0, minute=0, second=0, microsecond=0).astimezone(tz=timezone.utc)
day_date



# Direct Analysis

In [ ]:
from pyphocorehelpers.plotting.image_plotting_helpers import IMShowHelpers



def plot_matrix(xbin_edges, ybin_edges, matrix, ax, **kwargs):

    def setup_stable_axes_limits(xbins_edges, ybin_edges, ax):
        " manually sets the axis data limits to disable autoscaling given the xbins_edges/ybin_edges "
        # x == horizontal orientation:
        ax.set_xlim(left=xbins_edges[0], right=xbins_edges[-1])
        ax.set_ylim(bottom=ybin_edges[0], top=ybin_edges[-1])


    variable_value = matrix

    xmin, xmax, ymin, ymax = (xbin_edges[0], xbin_edges[-1], ybin_edges[0], ybin_edges[-1]) # the same for both orientations
    x_first_extent = (xmin, xmax, ymin, ymax) # traditional order of the extant axes
    # y_first_extent = (ymin, ymax, xmin, xmax) # swapped the order of the extent axes.
    main_plot_kwargs = {
        'cmap': 'viridis',
        'origin':'lower',
        'extent':x_first_extent,
        'aspect': 'auto',        
    }

    """
    Note that changing the origin while keeping everything else the same doesn't flip the direction of the yaxis labels despite flipping the yaxis of the data.
    """
    im_out = ax.imshow(variable_value, **main_plot_kwargs)
    setup_stable_axes_limits(xbin_edges, ybin_edges, ax)
    return im_out




In [ ]:
fig = plt.figure(layout="constrained", figsize=[9, 9], dpi=220, clear=True) # figsize=[Width, height] in inches.
long_width_ratio = 1
ax_dict = fig.subplot_mosaic(
    [
        ["ax_dumb", "ax_dumb_avg"],
        ["ax_good", "ax_good_avg"],
		
        # ["ax_dumb_avg"],
        # ["ax_good_avg"],
    ],
    # set the height ratios between the rows
    # set the width ratios between the columns
    width_ratios=[10, 1],
    sharey=True,
    gridspec_kw=dict(wspace=0, hspace=0.0) # `wspace=0`` is responsible for sticking the pf and the activity axes together with no spacing
)
fig.show()

In [ ]:
import napari

viewer = napari.Viewer(ndisplay=3)

# viewer.add_points(mov_avg_filtered, size=2, face_color='red', name='mov_avg_filtered')
# viewer.add_points(data2, size=2, face_color='green', name='dataset2')
# viewer.add_points(data3, size=2, face_color='blue', name='dataset3')



In [ ]:

named_sessions_dict = {'dumb': -2,
					   'good': -6,
}

# dumb_session_idx: int = -2

_out_layers = {}

for a_name, an_xdf_dataset_idx in named_sessions_dict.items():
    a_raw = active_only_out_eeg_raws[an_xdf_dataset_idx]

    a_df = a_raw.annotations.to_data_frame(time_format='datetime')
    a_df = a_df[a_df['description'] != 'BAD_motion']

    a_result = results[an_xdf_dataset_idx]
    # a_stream_info = deepcopy(_out_xdf_stream_infos_df).loc[an_xdf_dataset_idx]    
    Sxx = a_result['spectogram']['Sxx']
    Sxx_avg = a_result['spectogram']['Sxx_avg']
    # t = a_result['spectogram']['t']
    # freqs = a_result['spectogram']['freqs']
    # fs = a_result['spectogram']['fs']
    freqs, t, _ = a_result['spectogram']['spectogram_result_dict']['AF3']
    
    Sxx_avg_across_channel_avg = np.atleast_2d(Sxx_avg.mean(dim='channels', skipna=True))
    # Sxx_avg_across_channel_avg
    
    Sxx_across_channel_avg = Sxx.mean(dim='channels', skipna=True)
    # Sxx_across_channel_avg

    ax_label = f"ax_{a_name}"
    ax_label_avg = f"ax_{a_name}_avg"
    # np.shape(Sxx_across_channel_avg)
    
    # xbin = deepcopy(t)
    # ybin = deepcopy(freqs)
    # xmin, xmax, ymin, ymax = (xbin[0], xbin[-1], ybin[0], ybin[-1])
    # # xmin, xmax, ymin, ymax = (active_one_step_decoder.ybin[0], active_one_step_decoder.ybin[-1], active_one_step_decoder.xbin[0], active_one_step_decoder.xbin[-1]) # Reversed x and y axes, seems not good.
    # extent = (xmin, xmax, ymin, ymax)
    
    # ax_dict[ax_label].imshow(Sxx_avg_across_channel_avg.T)
    # ax_dict[ax_label_avg].imshow(Sxx_across_channel_avg, extent=extent, origin='lower')
    
    # fig, axs, plot_im_out = IMShowHelpers.final_x_vertical_plot_imshow(xbin_edges=np.arange(1), ybin_edges=freqs, matrix=Sxx_avg_across_channel_avg, ax=ax_dict[ax_label])
    # ax_dict[ax_label].autoscale(False)
    # fig, axs, plot_im_out = IMShowHelpers.final_x_horizontal_plot_imshow(xbin_edges=t, ybin_edges=freqs, matrix=Sxx_across_channel_avg, ax=ax_dict[ax_label])
    im_out = plot_matrix(xbin_edges=t, ybin_edges=freqs, matrix=Sxx_across_channel_avg, ax=ax_dict[ax_label])
    ax_dict[ax_label].set_ylabel(a_name)
    # ax_dict[ax_label].autoscale(False)
    

    # fig, axs, plot_im_out = IMShowHelpers.final_x_vertical_plot_imshow(xbin_edges=t, ybin_edges=freqs, matrix=Sxx_avg_across_channel_avg, ax=ax_dict[ax_label_avg])
    avg_im_out = plot_matrix(xbin_edges=[0.0, 1.0], ybin_edges=freqs, matrix=Sxx_avg_across_channel_avg, ax=ax_dict[ax_label_avg])
    # ax_dict[ax_label_avg].set_ylabel(f"{a_name}_avg")

    
    napari_img_layer_kwargs = dict(
        # channel_axis=None,
        # rgb=None,
        colormap='yellow', #'bop_blue',
        # contrast_limits=None,
        gamma=0.02,
        interpolation2d='nearest',
        interpolation3d='linear',
        rendering='mip',
        depiction='volume',
        # iso_threshold=None,
        # attenuation=0.05,
        # name=None,
        # metadata=None,
        # scale=None,
        translate=None,
        # rotate=None,
        # shear=None,
        # affine=None,
        # opacity=1,
        blending='additive',
        visible=True,
        # multiscale=None,
        # cache=True,
        # plane=None,
        # experimental_clipping_planes=None,
        # custom_interpolation_kernel_2d=None,
    )
    

    curr_name = f"{a_name}"
    _out_layers[curr_name] = viewer.add_image(Sxx, name=curr_name, **napari_img_layer_kwargs)
    viewer.dims.axis_labels = Sxx.dims # ('channels', 'freqs', 'times')

    curr_name = f"{a_name}_avg"
    # napari_img_layer_kwargs['translate'] = [0.0, 1852.0]
    napari_img_layer_kwargs['translate'] = [0.0, -2.0]
    _out_avg_layer = viewer.add_image(Sxx_avg.T, name=curr_name, **napari_img_layer_kwargs) #  colormap='bop_blue', gamma=0.20, rendering='additive'
    _out_layers[curr_name] = _out_avg_layer


In [ ]:
# _out_avg_layer.translate([512.0, 0.0])
# _out_avg_layer.set_translation([512.0, 0.0])
# _out_avg_layer.translate = [0.0, 513.0]
_out_avg_layer.translate = [0.0, 1852.0]
_out_avg_layer.translate

In [ ]:
_out_avg_layer.

In [ ]:
viewer.dims
viewer.layers

In [ ]:
from pyphoplacecellanalysis.GUI.Napari.napari_helpers import napari_extract_layers_info

# @function_attributes(short_name=None, tags=['napari', 'config'], input_requires=[], output_provides=[], uses=[], used_by=['napari_extract_layers_info'], creation_date='2024-08-12 08:54', related_items=[])
def extract_layer_info(a_layer):
    """ Extracts info as a dict from a single Napari layer. 
    by default Napari layers print like: `<Shapes layer 'Shapes' at 0x1635a1e8460>`: without any properties that can be easily referenced.
    This function extracts a dict of properties.

    from pyphoplacecellanalysis.GUI.Napari.napari_helpers import extract_layer_info

    """
    out_properties_dict = {}
    positioning = ['scale', 'translate', 'rotate', 'shear', 'affine', 'corner_pixels']
    visual = ['opacity', 'blending', 'visible', 'z_index', 'contrast_limits_range', '_colormap_name', 'gamma']
    # positioning = ['scale', 'translate', 'rotate', 'shear', 'affine']
    out_properties_dict['positioning'] = {}

    for a_property_name in positioning:
        out_properties_dict['positioning'][a_property_name] = getattr(a_layer, a_property_name)

    out_properties_dict['visual'] = {}
    for a_property_name in visual:
        try:
            out_properties_dict['visual'][a_property_name] = getattr(a_layer, a_property_name)
        except (AttributeError, ValueError, KeyError, TypeError) as e:
            print(f'failed to get property: "{a_property_name}" with error {e}')
            pass
        except Exception as e:
            raise

    return out_properties_dict


# @function_attributes(short_name=None, tags=['napari', 'config'], input_requires=[], output_provides=[], uses=['extract_layer_info'], used_by=[], creation_date='2024-08-12 08:54', related_items=[])
def napari_extract_layers_info(layers):
	"""extracts info dict from each layer as well.
	Usage:
        from pyphoplacecellanalysis.GUI.Napari.napari_helpers import napari_extract_layers_info
		layers = directional_viewer.layers # [<Shapes layer 'Shapes' at 0x1635a1e8460>, <Shapes layer 'Shapes [1]' at 0x164d5402e50>]
		out_layers_info_dict = debug_print_layers_info(layers)

	"""
	out_layers_info_dict = {}
	for a_layer in layers:
		a_name: str = str(a_layer.name)
		out_properties_dict = extract_layer_info(a_layer)
		out_layers_info_dict[a_name] = out_properties_dict
		# if isinstance(a_layer, Shapes):
		# 	print(f'shapes layer: {a_layer}')
		# 	a_shapes_layer: Shapes = a_layer
		# 	# print(f'a_shapes_layer.properties: {a_shapes_layer.properties}')
		# 	out_properties_dict = extract_layer_info(a_layer)
		# 	print(f'\tout_properties_dict: {out_properties_dict}')
		# 	out_layers_info_dict[a_name] = out_properties_dict
		# else:
		# 	print(f'unknown layer: {a_layer}')	
	return out_layers_info_dict





layers = viewer.layers # [<Shapes layer 'Shapes' at 0x1635a1e8460>, <Shapes layer 'Shapes [1]' at 0x164d5402e50>]
out_layers_info_dict = napari_extract_layers_info(layers)
out_layers_info_dict

In [ ]:
out_layers_info_dict['dumb']

In [ ]:
from napari.layers.image.image import Image

property_names = ['metadata', 'blending', 'opacity', 'rendering', 'scale', 'gamma', 'contrast_limits_range', 'colormap']
for a_layer in layers:
    an_img_layer: Image = a_layer
    
#    type(a_layer)

# an_img_layer.blending
an_img_layer.__dict__

In [ ]:
napari.run()

In [ ]:
xbin
ybin

In [ ]:
a_raw.plot()

In [ ]:
a_stream_info

In [ ]:
a_raw

In [ ]:
import xarray as xr

## INPUTS: a_result
a_spectogram_result: Dict = a_result['spectogram'] 

ch_names = a_spectogram_result['ch_names']
fs = a_spectogram_result['fs']
a_spectogram_result_dict = a_spectogram_result['spectogram_result_dict'] # Dict[channel: Tuple]
Sxx = a_spectogram_result['Sxx']
Sxx_avg = a_spectogram_result['Sxx_avg']

Sxx

In [ ]:

Sxx_avg_list = [] 

# ch_names = a_raw.info.ch_names

for a_ch, a_tuple in a_spectogram_result_dict.items():
    f, t, Sxx = a_tuple ## unpack the tuple
    # np.shape(Sxx) # (513, 1116) - (n_freqs, n_times)
    n_freqs = np.shape(f)
    n_times = np.shape(t) 
    Sxx_avg = np.nanmean(Sxx, axis=-1) ## average over all time to get one per session
    Sxx_avg_list.append(Sxx_avg)
    
Sxx_avg_list = np.stack(Sxx_avg_list) # (14, 513) - (n_channels, n_freqs)
Sxx_avg_list = xr.DataArray(Sxx_avg_list, dims=("channels", "freqs"), coords={"channels": ch_names, "freqs": f})
np.shape(Sxx_avg_list)
Sxx_avg_list

In [ ]:
ch_names

In [ ]:
a_raw.annotations

In [ ]:
Sxx_avg = np.nanmean(Sxx, axis=-1) ## average over all time to get one per session
Sxx_avg

In [ ]:
plt.close('all')

In [ ]:
plot_all_spectograms(active_only_out_eeg_raws, results)

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["axes.titlesize"] = 8
plt.rcParams["axes.labelsize"] = 8
plt.rcParams["xtick.labelsize"] = 6
plt.rcParams["ytick.labelsize"] = 6
plt.rcParams["legend.fontsize"] = 6
plt.rcParams["figure.titlesize"] = 8
plt.rcParams["axes.titlepad"] = 0
plt.rcParams["figure.constrained_layout.use"] = True
plt.rcParams["figure.constrained_layout.h_pad"] = 0.0
plt.rcParams["figure.constrained_layout.w_pad"] = 0.0
plt.rcParams["figure.constrained_layout.hspace"] = 0.0
plt.rcParams["figure.constrained_layout.wspace"] = 0.0
plt.rcParams["figure.subplot.wspace"] = 0.0
plt.rcParams["figure.subplot.hspace"] = 0.0
plt.rcParams["figure.subplot.wspace"] = 0.0
plt.rcParams["figure.subplot.hspace"] = 0.0


In [ ]:
## Plot a synchronized EEG Raw data and Spectogram Figure together:
active_eeg_idx: int = -4
mne_raw_fig = active_only_out_eeg_raws[active_eeg_idx].plot(time_format='datetime', scalings='auto') # MNEBrowseFigure
fig, axs = plot_session_spectogram(active_only_out_eeg_raws[active_eeg_idx], results[active_eeg_idx], sync_to_mne_raw_fig=mne_raw_fig)
# plt.subplots_adjust(wspace=0, hspace=0)  # remove spacing


In [ ]:
active_eeg_idx: int = -6
mne_raw_fig2 = active_only_out_eeg_raws[active_eeg_idx].plot(time_format='datetime', scalings='auto') # MNEBrowseFigure
fig2, axs2 = plot_session_spectogram(active_only_out_eeg_raws[active_eeg_idx], results[active_eeg_idx], sync_to_mne_raw_fig=mne_raw_fig)

### Compute and show all spectograms

In [ ]:
# Compute results first
active_only_out_eeg_raws, results = batch_compute_all_eeg_datasets(eeg_raws=_out_eeg_raw, limit_num_items=3)

# Render to PDFs (paged)
from pathlib import Path
out_paths = render_all_spectograms_to_high_quality_pdfs(
    active_only_out_eeg_raws,
    results,
    output_parent_folder=Path(r"E:/Dropbox (Personal)/Databases/AnalysisData/MNE_preprocessed/exports"),
    mode="paged",
    seconds_per_page=180.0,
    freq_min_hz=1.0,
    freq_max_hz=40.0,
    dpi=300
)
print(f"Wrote {len(out_paths)} PDF(s)")

## ANalysis for Fatigue/Bandpowers

In [ ]:
from phoofflineeeganalysis.analysis.computations.fatigue_analysis import compare_multiple_recordings, compute_fatigue_metrics, analyze_fatigue_trends, print_analysis_report, visualize_fatigue_comparison

raw_objects = deepcopy(_out_eeg_raw)
raw_obj_labels = [str(a_raw) for a_raw in raw_objects]
if len(raw_objects) >= 2:
    results = compare_multiple_recordings(raw_objects, raw_obj_labels[:len(raw_objects)])
    print_analysis_report(results)
    visualize_fatigue_comparison(results)

    results
else:
    print("Not enough data files found for comparison.")


## Sliding wavlet analyses

In [ ]:
import pyqtgraph as pg
from phoofflineeeganalysis.timeflux.nodes.wavelet_cwt import EEGViewer

app = pg.mkQApp('EEGWaveletViewer')

raw = _out_eeg_raw[-1]

# Launch the Qt Application
viewer = EEGViewer(raw_data=raw)
viewer.show()
# sys.exit(app.exec_())


In [ ]:
fixed_len_epochs

In [ ]:
from phoofflineeeganalysis.timeflux.nodes.wavelet_cwt import plot_raw_with_cwt

# --- Example Usage ---

# First, create a sample 14-channel Raw object for demonstration
# sfreq = 250
# ch_names = [f'EEG {i+1:02d}' for i in range(14)]
# info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')
# n_seconds = 120
# data = np.random.randn(len(ch_names), sfreq * n_seconds)
# raw = mne.io.RawArray(data, info)

raw = _out_eeg_raw[-1]

# Now, use the function to plot the first channel (index 0) from 10 to 20 seconds
plot_raw_with_cwt(raw, start_seconds=10.0, end_seconds=20.0, channel_index=0)
